### Experiment with a local llm as judge, critic etc'
* Need to use code from elsewhere for munging candidates into prompts
* Open Medical-LLM Leaderboard: https://huggingface.co/spaces/openlifescienceai/open_medical_llm_leaderboard
* Need to try diff models
* https://www.reddit.com/r/LocalLLaMA/comments/1cec23f/llama3_based_openbiollm70b_8b_outperforms_gpt4/
*  https://huggingface.co/aaditya/Llama3-OpenBioLLM-8B

In [ ]:
from cmim import CMIMFeatureSelector
# cmim = CMIMFeatureSelector(task='classification')


In [ ]:
# ## https://colab.research.google.com/drive/1F5oV20InEYeAJGmBwYF9NM_QhLmjBkKJ?usp=sharing#scrollTo=fca6d8cfwOUO
# # !CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python
# !pip install llama-cpp-python

In [24]:
# !pip install torch torchvision transformers -U
# !pip3 install torch torchvision torchaudio transformers sentence-transformers -U
# !conda uninstall tokenizers -y
# !pip3 install tokenizers transformers -U
# !pip3 install safetensors
# !pip3 install accelerate datasets peft trl bitsnbytes -U 

In [25]:
import transformers
import torch
import pandas as pd
from tqdm import tqdm
import re
from sentence_transformers import SentenceTransformer
import numpy as np

ANNOT_DATA = "../.././Reports/example_plausible_gallstones.csv"

In [26]:
# # https://colab.research.google.com/drive/1F5oV20InEYeAJGmBwYF9NM_QhLmjBkKJ?usp=sharing#scrollTo=2xawFMf70x7F
# from huggingface_hub import hf_hub_download
# from llama_cpp import Llama
# 
# model_name = "aaditya/OpenBioLLM-Llama3-8B-GGUF"
# model_file = "openbiollm-llama3-8b.Q5_K_M.gguf"
# 
# model_path = hf_hub_download(model_name,
#                              filename=model_file,
#                              # local_dir='/content
#                              )
# print("My model path: ", model_path)
# llm = Llama(model_path=model_path,
#             n_gpu_layers=-1)

In [27]:
# Question = "How can i split a 3mg or 4mg waefin pill so i can get a 2.5mg pill?"
# 
# #ORIG:
# # prompt = f"You are an expert and experienced from the healthcare and biomedical domain with extensive medical knowledge and practical experience. Your name is OpenBioLLM, and you were developed by Saama AI Labs with Open Life Science AI. who's willing to help answer the user's query with explanation. In your explanation, leverage your deep medical expertise such as relevant anatomical structures, physiological processes, diagnostic criteria, treatment guidelines, or other pertinent medical concepts. Use precise medical terminology while still aiming to make the explanation clear and accessible to a general audience. Medical Question: {Question} Medical Answer:"
# #ALT:
# prompt = f"""You are an expert and experienced researcher from the healthcare and biomedical domain with extensive medical knowledge and practical experience. Willing to help answer the user's query with explanation. 
# In your explanation, leverage your deep medical expertise such as relevant anatomical structures, physiological processes, diagnostic criteria, treatment guidelines, or other pertinent medical concepts and scientific literature.
# Use precise terminology while still aiming to make the explanation clear and accessible. Medical Question: {Question} Medical Answer:"""
# 
# response = llm(prompt, max_tokens=64)['choices'][0]['text']
# 
# print("\n\n\n", response)

In [ ]:
# model_id = "aaditya/OpenBioLLM-Llama3-8B"

# pipeline = transformers.pipeline(
#     "text-generation",
#     model=model_id,
#     model_kwargs={"torch_dtype": torch.bfloat16},
#     device="mps"
#     # "auto",
# )

# ## I changed prompt
# messages = [
#     {"role": "system", "content": "You are an expert and experienced researcher from the healthcare and biomedical domain with extensive medical knowledge and practical experience. Willing to help answer the user's query with explanation. In your explanation, leverage your deep medical expertise such as relevant anatomical structures, physiological processes, diagnostic criteria, treatment guidelines, or other pertinent medical concepts and scientific literature. Use precise terminology while still aiming to make the explanation clear and accessible."},
#     {"role": "user", "content": "How can i split a 3mg or 4mg waefin pill so i can get a 2.5mg pill?"},
# ]

# prompt = pipeline.tokenizer.apply_chat_template(
#         messages, 
#         tokenize=False, 
#         add_generation_prompt=True
# )

# terminators = [
#     pipeline.tokenizer.eos_token_id,
#     pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
# ]

# outputs = pipeline(
#     prompt,
#     max_new_tokens= 64#256,
#     ,eos_token_id=terminators,
#     do_sample=True,
#     temperature=0.0
#     ,top_p=0.9,
# )
# print(outputs[0]["generated_text"][len(prompt):])


### Load existing annotated examples and get inference on them
* 0-shot.
* Could use embeddings? 


In [ ]:
df = pd.read_csv(ANNOT_DATA)
## Could filter by p-values or uncommented cases
DROP_UNCOMMENTED_EXAMPLES = False

data = df.drop_duplicates(subset=df.select_dtypes(["O","bool"]).columns)
print(data.shape[0])
if DROP_UNCOMMENTED_EXAMPLES:
    data = data.loc[~data["COMMENTS"].isna()].copy()
    print(data.shape[0])
# #Drop more cases - context limit, or for test set
# data = data.loc[~data['COMMENTS'].str.contains("P val|p-val|correlation|ordinal|similar to",case=False,na=False)]
# print(data.shape[0])
# # data = data.drop_duplicates(subset=['feature_name']).drop_duplicates(subset=['COMMENTS']) ## 31 -> 27
# data = data.drop_duplicates(subset=['feature_name','COMMENTS']) ## 
print(data.shape)
display(data)

## Load updated candidates (containing more metadata) and see if can match it to the annotations

In [ ]:
# df_update = pd.read_csv("gallstone_review_interesting_candidates_results.csv")
# gallstone_ipw_broad_feature_report.csv
df_update = pd.read_csv("../../broad_candidate_novel_cuis_chol.csv")
print(df_update.shape[0]) ## multiple rows per feature 
numeric_cols = df_update.select_dtypes(include='number').columns
# df_update = df_update.drop_duplicates(["feature_name"])
df_update = df_update.groupby(["feature_name"])[numeric_cols].max()#min()
print(df_update.shape[0])
df_update

In [ ]:
# print(df_update.merge(data,on="feature_name",how="right", suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)').shape)
s1 = data.shape[0]
data = data.merge(df_update,on="feature_name",how="left", suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
assert s1 == data.shape[0]


In [ ]:
print("Corr")
data.select_dtypes(["number","bool"]).corr().round(2)[["Interesting?","Plausible/Makes Sense?"]].dropna(axis=0).sort_values("Interesting?")

In [ ]:
def get_examples(data,target,get_interesting=True,get_plausible=True,print_plausible_prompt=False):
    results = []

    novel_prompts = []
    plausible_prompts = []
    for index, row in tqdm(data.iterrows(), total=data.shape[0], desc="Processing rows"):
        feature_name = row['feature_name']
        target = row['Target'].replace(r"\(\)","").replace(" AND "," ").replace(r"(","").replace(r")","")
        # target = "Cholelithiasis OR Gallstones OR Gallbladder disease OR cholecystitis"
        p_val = row['p_val']
        feature_importance = row['feature_importance']
        correlation = row['correlation']
        ## related to biomedical and healthcare disease understanding, prognosis, research, and prediction of:
        prompt_interesting = f"""Given the feature '{feature_name}'; (p-value: {p_val}, correlation: {correlation}, SHAP importance {feature_importance}), as a potential predictive feature, or risk/protective factor for target disease: '{target}', is this feature Novel and interesting?
       \Output Yes or No prediction""" #,then explain why for a medical expert."""
        
        prompt_plausible = f"""Given the feature '{feature_name}'; (p-value: {p_val}, correlation: {correlation}, SHAP importance {feature_importance}), as a potential predictive feature, or risk/protective factor for target disease:'{target}',
        does this feature make sense? Is there a plausible mechanism for the feature to affect the target?\
        Output Yes or No prediction""" #, then explain for a medical expert.
        
        if print_plausible_prompt:
            print(prompt_plausible)
            print()

        novel_prompts.append(prompt_interesting)
        plausible_prompts.append(prompt_plausible)
    
    assert len(novel_prompts)==len(plausible_prompts) ## Doesn't check for nans or empty
    return novel_prompts,plausible_prompts

        ## following part was used for making training examples or ICL examples ofr GPT. comment out for now !
    #     response_interesting = f"{'Yes' if row['Interesting?'] else 'No'}. {'' if pd.isna(row['COMMENTS']) else row['COMMENTS'].strip()}"

    #     response_plausible = f"{'Yes' if row['Plausible/Makes Sense?'] else 'No'}."
    #     # # if not row['COMMENTS'].str.contains("P value|correlation",case=False,na=False):
    #     # if not row['COMMENTS'].contains("P value|correlation",case=False,na=False):
    #         # response_plausible = response_plausible + f" {row['COMMENTS'].str.strip()}"
    #     # Check if 'COMMENTS' does not contain "P value" or "correlation"
    #     if pd.notna(row['COMMENTS']) and not re.search("P value|correlation", row['COMMENTS'], flags=re.IGNORECASE):
    #         response_plausible += f" {row['COMMENTS'].strip()}"    
    #     if get_interesting:
    #         results.append( {"role": "user", "content":prompt_interesting})
    #         results.append( {"role": "assistant", "content":response_interesting})
    #     if get_plausible:
    #         results.append( {"role": "user", "content":prompt_plausible})
    #         results.append( {"role": "assistant", "content":response_plausible})
    # return results

In [ ]:
novel_prompts,plausible_prompts = get_examples(data,target = "Cholelithiasis OR Gallstones OR Gallbladder disease OR cholecystitis")
# results

In [ ]:
# novel_prompts

In [ ]:
# %%time
# novel_res = []
# for i in novel_prompts[0:5]:
#     novel_res.append(llm(i, max_tokens=64)['choices'][0]['text'])
# novel_res

### Let's try a naive embedding + train a model on that
* MTEB sota lb are mostly instruct tuned embedder oriented. Hope it won't mess this up.
* Could use the llama models for text gen, but for a pure number - maybe easier to get embeds (faster than finetuning)

* https://huggingface.co/Alibaba-NLP/gte-Qwen2-7B-instruct
    * Lots of files?
* https://huggingface.co/bijaygurung/stella_en_400M_v5
    * Smaller model. More instruct tuning seemingly?  
* https://huggingface.co/spaces/mteb/leaderboard


Could try SetFit finetuning? (Few shot learning)
* https://huggingface.co/blog/setfit
* https://huggingface.co/docs/setfit/quickstart

In [ ]:
# 

# ## https://huggingface.co/Alibaba-NLP/gte-Qwen2-7B-instruct
# model = SentenceTransformer("Alibaba-NLP/gte-Qwen2-7B-instruct", trust_remote_code=True ) # enabled originally
# # In case you want to reduce the maximum length:
# model.max_seq_length = 4096#8192

# queries = [
#     "how much protein should a female eat",
#     "summit define",
# ]
# documents = [
#     "As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
#     "Definition of summit for English Language Learners. : 1  the highest point of a mountain : the top of a mountain. : 2  the highest level. : 3  a meeting or series of meetings between the leaders of two or more governments.",
# ]

# query_embeddings = model.encode(queries, prompt_name="query")
# document_embeddings = model.encode(documents)

# scores = (query_embeddings @ document_embeddings.T) * 100
# print(scores.tolist())


In [ ]:
from sentence_transformers import SentenceTransformer

# # This model supports two prompts: "s2p_query" and "s2s_query" for sentence-to-passage and sentence-to-sentence tasks, respectively.
# # They are defined in `config_sentence_transformers.json`
# query_prompt_name = "s2p_query"
# queries = [
#     "What are some ways to reduce stress?",
#     "What are the benefits of drinking green tea?",
# ]
# # docs do not need any prompts
# docs = [
#     "There are many effective ways to reduce stress. Some common techniques include deep breathing, meditation, and physical activity. Engaging in hobbies, spending time in nature, and connecting with loved ones can also help alleviate stress. Additionally, setting boundaries, practicing self-care, and learning to say no can prevent stress from building up.",
#     "Green tea has been consumed for centuries and is known for its potential health benefits. It contains antioxidants that may help protect the body against damage caused by free radicals. Regular consumption of green tea has been associated with improved heart health, enhanced cognitive function, and a reduced risk of certain types of cancer. The polyphenols in green tea may also have anti-inflammatory and weight loss properties.",
# ]

# # ！The default dimension is 1024, if you need other dimensions, please clone the model and modify `modules.json` to replace `2_Dense_1024` with another dimension, e.g. `2_Dense_256` or `2_Dense_8192` !
# # on gpu
# model = SentenceTransformer("dunzhang/stella_en_400M_v5", trust_remote_code=True)#.cuda() # needs xformers
# # you can also use this model without the features of `use_memory_efficient_attention` and `unpad_inputs`. It can be worked in CPU.
# # model = SentenceTransformer(
# #     "dunzhang/stella_en_400M_v5",
# #     trust_remote_code=True,
# #     device="cpu",
# #     config_kwargs={"use_memory_efficient_attention": False, "unpad_inputs": False}
# # )
# query_embeddings = model.encode(queries, prompt_name=query_prompt_name)
# doc_embeddings = model.encode(docs)
# print(query_embeddings.shape, doc_embeddings.shape)
# # (2, 1024) (2, 1024)
# similarities = model.similarity(query_embeddings, doc_embeddings)

In [ ]:
# model = SentenceTransformer("BAAI/bge-small-en-v1.5", trust_remote_code=True)#.cuda()
model = SentenceTransformer("BAAI/bge-base-en-v1.5", trust_remote_code=True) #  53,66 auc (Without features)
# model = SentenceTransformer("FremyCompany/BioLORD-2023")

# model = SentenceTransformer("Alibaba-NLP/gte-large-en-v1.5", trust_remote_code=True)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import RocCurveDisplay
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.impute import SimpleImputer

In [ ]:
%%time
X = model.encode(novel_prompts)
X.shape

In [ ]:
display(data.select_dtypes(["number"])) # Lacks mos useful metadata like path length

In [ ]:
# X = np.hstack([X,data.select_dtypes(["number"]).fillna(-0.1)]) ## Massively boost performance
X = np.hstack([X,data.select_dtypes(["number"])]) #keep nans - if rf
# from 53,67 (w'BAAI/bge-base-en-v1.5) to 61, 74 auc  (with basic min feats)
print(X.shape)

In [ ]:
# clf = LogisticRegressionCV()
# clf = LogisticRegression()
# clf = make_pipeline(StandardScaler(),SimpleImputer(add_indicator=True), LogisticRegression())
clf = RandomForestClassifier(500, min_samples_split=4, min_samples_leaf=3,ccp_alpha=0.015) ## 60, 66 (with extra feats; max)
# clf = KNeighborsClassifier().8

In [ ]:
print("Novel")
y_novel = data["Interesting?"].astype(int)
y_novel_pred = cross_val_predict(clf,X,y_novel)
y_proba = cross_val_predict(clf,X,y_novel,method="predict_proba")[:,1]
print(classification_report(y_novel,y_novel_pred))
print("rocAUC",round(roc_auc_score(y_novel,y_proba),3))

In [ ]:
RocCurveDisplay.from_predictions( y_novel,y_proba,plot_chance_level=True)

In [ ]:
print("Plausible")
y_plausible = data["Plausible/Makes Sense?"].astype(int)
y_plausible_pred = cross_val_predict(clf,X,y_plausible)
print(classification_report(y_plausible,y_plausible_pred))
y_proba = cross_val_predict(clf,X,y_novel,method="predict_proba")[:,1]

print("rocAUC",round(roc_auc_score(y_plausible,y_proba),3))
RocCurveDisplay.from_predictions( y_plausible,y_proba,plot_chance_level=True)